In [3]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import spacy
import nltk
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes, svm, linear_model
from sklearn.metrics import accuracy_score

import xgboost
# nltk.download('stopwords')

from sklearn.pipeline import Pipeline

from gensim.models import Word2Vec, KeyedVectors

np.random.seed(359)

## Preprocessing the Text

In [5]:
# len(stopwords.words("german"))

de = spacy.load('de_core_news_sm')
stopwords = de.Defaults.stop_words

In [6]:
data = pd.read_excel(io="../data/data2020.xlsx",
                     usecols=['Politikbereich', 'Zweck'])

umlauts = {'ä': 'ae', 'ö': 'oe', 'ü': 'ue'}

In [7]:
data['Zweck_clean'] = data['Zweck'].str.replace('[^\w\s]+', ' ', regex=True) \
                             .str.replace('\d+', ' ', regex=True) \
                             .str.replace(' +', ' ', regex=True) \
                             .replace(umlauts, regex=True) \
                             .str.lower() \
                             .apply(lambda x: [word.lemma_ for word in de(x)]) \
                             .apply(lambda x: [item for item in x if item not in stopwords]) \
                             .str.join(' ')
                             

In [8]:
data['Zweck_clean'].head(10)

0                         nord sued tangente linie bpu
1    fit fuer schule projekt foerderung sprachentwi...
2                  kontakt beratungsstelle sleepin kub
3    kita gesetz ausgestaltung paedagogischer raeum...
4    ausstellungen schoener strom einzug elektrizit...
5    personal sachkosten fuer öffentlichkeitsarbeit...
6                                    ladeinfrastruktur
7       apg projektnr apg mh projekt janz nah draussen
8            radfahrtrainings fuer gefluechtete frauen
9                           errichtung betriebsstaette
Name: Zweck_clean, dtype: object

In [9]:
data['Zweck_clean'].str.len().value_counts()

40     687
21     531
15     463
8      373
55     350
      ... 
237      1
169      1
204      1
208      1
148      1
Name: Zweck_clean, Length: 213, dtype: int64

In [10]:
data['Zweck'].head(10)

0      Nord-Süd-Tangente; Linie 26/27, 2.2. Teil-BPU,A
1    Zusammen Fit für die Schule – Projekt zur Förd...
2           Kontakt- und Beratungsstelle/SleepIn (KuB)
3    Gute-Kita-Gesetz- Ausgestaltung pädagogischer ...
4    Ausstellungen Schöner Strom/Einzug der Elektri...
5    Personal- und Sachkosten für Öffentlichkeitsar...
6                                    Ladeinfrastruktur
7    APG 0920-68406-353 / Projektnr.  APG-MH/1-2020...
8              Radfahrtrainings für geflüchtete Frauen
9                      Errichtung einer Betriebsstätte
Name: Zweck, dtype: object

In [11]:
train_X, test_X, train_y, test_y = train_test_split(data['Zweck_clean'], 
                                                    data['Politikbereich'], 
                                                    test_size=0.2)

encoder = LabelEncoder()
encoder = encoder.fit(data['Politikbereich'])
train_y_enc = encoder.transform(train_y)
test_y_enc = encoder.transform(test_y)

In [12]:
Tfidf_vect = TfidfVectorizer(max_features=2000)
Tfidf_vect.fit(data['Zweck_clean'])
Train_X_Tfidf = Tfidf_vect.transform(train_X)
Test_X_Tfidf = Tfidf_vect.transform(test_X)

In [14]:
train_X

3765     unterstuetzung aufbauphase existenzgruendungen...
7453     herstellungstechnologien replizierten stark ge...
7439         lohnkostenzuschuss fuer blueprint berlin gmbh
8466                        stadt kiez schule zeig zukunft
5953     bezirkliche stadtteilkoordination sicherung st...
                               ...                        
7636       frei raeume fuer maedchen junge frauen weiß ...
4359     ergaenzung streetwork aufsuchend jugendsoziala...
10027                             arbeitsintegration aktiv
7910                                           plug hybrid
5092     ehap ii aha ehap beratung reinickendorf foerde...
Name: Zweck_clean, Length: 9015, dtype: object

In [13]:
train_X

3765     unterstuetzung aufbauphase existenzgruendungen...
7453     herstellungstechnologien replizierten stark ge...
7439         lohnkostenzuschuss fuer blueprint berlin gmbh
8466                        stadt kiez schule zeig zukunft
5953     bezirkliche stadtteilkoordination sicherung st...
                               ...                        
7636       frei raeume fuer maedchen junge frauen weiß ...
4359     ergaenzung streetwork aufsuchend jugendsoziala...
10027                             arbeitsintegration aktiv
7910                                           plug hybrid
5092     ehap ii aha ehap beratung reinickendorf foerde...
Name: Zweck_clean, Length: 9015, dtype: object

In [11]:
Train_X_Tfidf

<9015x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 27260 stored elements in Compressed Sparse Row format>

In [12]:
model = KeyedVectors.load_word2vec_format('/Users/boris/Downloads/german.model', binary=True)

In [13]:
train_X

3765     unterstuetzung aufbauphase existenzgruendungen...
7453     herstellungstechnologien replizierten stark ge...
7439         lohnkostenzuschuss fuer blueprint berlin gmbh
8466                        stadt kiez schule zeig zukunft
5953     bezirkliche stadtteilkoordination sicherung st...
                               ...                        
7636       frei raeume fuer maedchen junge frauen weiß ...
4359     ergaenzung streetwork aufsuchend jugendsoziala...
10027                             arbeitsintegration aktiv
7910                                           plug hybrid
5092     ehap ii aha ehap beratung reinickendorf foerde...
Name: Zweck_clean, Length: 9015, dtype: object

In [14]:
model.most_similar(positive=['Koenig', 'Frau'], negative=['Mann'])

[('Koenigin', 0.7524535655975342),
 ('Prinzessin', 0.7145279049873352),
 ('Prinz', 0.6881615519523621),
 ('Jungschuetzenkoenigin', 0.6740390658378601),
 ('Majestaet', 0.6590647101402283),
 ('Kronprinz_Philippe', 0.6576778292655945),
 ('Regentin', 0.6576672196388245),
 ('Prinzen', 0.6576202511787415),
 ('Carl_XVI', 0.6540848612785339),
 ('Prinzessin_Maxima', 0.6527888774871826)]

In [15]:
my_dict = dict({})
for idx, key in enumerate(model.index_to_key):
    my_dict[key] = model.get_vector(key)

In [31]:
def vectorize(X):
    vecs = np.array([])
    for words in X:
        mean_vec = np.array([])
        for w in words:
            if w in my_dict:
                np.append(mean_vec, my_dict[w], axis = 0)
            elif w.capitalize() in my_dict:
                np.append(mean_vec, my_dict[w.capitalize()], axis = 0)
            else:
                np.append(mean_vec, np.zeros(300), axis = 0)       
            
        np.append(vecs, np.mean(mean_vec, axis=0), axis = 0)
        
        
    return vecs

In [32]:
Train_X_Word2Vec = vectorize(train_X)
Test_X_Word2Vec = vectorize(test_X)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 0 dimension(s)

In [30]:
Train_X_Word2Vec

array([], dtype=float64)

## Modelling

In [18]:
xg = xgboost.XGBClassifier(use_label_encoder=False, eval_metric="merror")
xg.fit(Train_X_Tfidf, train_y_enc)

predictions_xg = xg.predict(Test_X_Tfidf)

print(accuracy_score(predictions_xg, test_y_enc))

0.6663708961845608


In [19]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf, train_y_enc)

predictions_NB = Naive.predict(Test_X_Tfidf)

print(accuracy_score(predictions_NB, test_y_enc))

0.6592724046140195


In [20]:
SVM = svm.SVC(C=1, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf, train_y_enc)

predictions_SVM = SVM.predict(Test_X_Tfidf)

print(accuracy_score(predictions_SVM, test_y_enc))

0.7191659272404614


In [21]:
LG = linear_model.LogisticRegression(max_iter=200)
LG.fit(Train_X_Tfidf, train_y_enc)

predictions_LG = LG.predict(Test_X_Tfidf)

print(accuracy_score(predictions_LG, test_y_enc))

0.707187222715173


In [22]:
SVM = svm.SVC(C=1, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Word2Vec, train_y_enc)

predictions_SVM = SVM.predict(Test_X_Word2Vec)

print(accuracy_score(predictions_SVM, test_y_enc))

0.37222715173025733


In [23]:
data.Politikbereich.value_counts()

Wirtschaft                                            2536
Arbeit                                                1889
Sport                                                 1121
Jugend                                                 874
Kultur                                                 829
Soziales                                               557
Bildung                                                520
Gesundheit                                             442
Bürgerschaftliches Engagement, Bürgerbeteiligung       404
Stadtentwicklung                                       303
Integration                                            301
Familie                                                224
Umwelt                                                 216
Gleichstellung                                         210
Verkehr                                                205
Antidiskriminierung                                    140
Pflege                                                  

In [24]:
print(2536/len(data))

0.22504215103380956


In [25]:
keys = data.Politikbereich.unique()